In [2]:
import auto_everything

In [4]:
!python3 -m pip install --upgrade Pillow

  Using cached Pillow-8.3.1-cp39-cp39-macosx_10_10_x86_64.whl (2.9 MB)
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [3]:
from PIL import Image
from io import BytesIO
from base64 import b64decode

from auto_everything.disk import Disk
disk = Disk()


class MyPillow():
    def read_image_from_bytes_io(self, bytes_io):
        return Image.open(bytes_io)

    def save_image_to_file_path(self, image, file_path):
        """
        foamat = [jpeg, png]
        """
        image.save(file_path)

    def get_image_bytes_size(self, image):
        image = image.convert('RGB')
        out = BytesIO()
        image.save(out, format="jpeg")
        return out.tell()

    def decrease_the_size_of_an_image(self, image, quality=None):
        image = image.convert('RGB')
        out = BytesIO()
        if quality is None:
            image.save(out, format="jpeg")
        else:
            image.save(out, format="jpeg", optimize=True, quality=quality)
        out.seek(0)
        return out
    
    def get_file_size(self, path: str, level: str = "B", bytes_size: int = None) -> int:
        """
        Get file size in the unit of  B, KB, MB.
        Parameters
        ----------
        path: string
            the file path
        level: string
            B, KB, or MB
        bytes_size: int
            a number represent the file size in bytes level
        """
        if bytes_size is None:
            path = self._expand_user(path)
            file = Path(path)
            assert file.exists(), f"{path} is not exist!"
            bytes_size = file.stat().st_size
        if level == "B":
            return int('{:.0f}'.format(bytes_size))
        elif level == "KB":
            return int('{:.0f}'.format(bytes_size / float(1 << 10)))
        elif level == "MB":
            return int('{:.0f}'.format(bytes_size / float(1 << 20)))


    def force_decrease_image_file_size(self, image, limit_in_kb="1024"):
        """
        :param image: PIL image
        :param limit: kb
        :return: bytes_io
        """
        image = image.convert('RGB')
        OK = False
        quality = 100
        out = BytesIO()
        while (OK is False):
            out = BytesIO()
            image.save(out, format="jpeg", optimize=True, quality=quality)
            size = self.get_file_size(path=None, bytes_size=out.tell(), level="KB")
            quality -= 10
            if size <= limit_in_kb or quality <= 10:
                OK = True
        out.seek(0)
        return out

myPillow = MyPillow()

In [4]:
import base64
import re

def myCompress(base64string):
    try:
        image = myPillow.read_image_from_bytes_io(BytesIO(base64.b64decode(base64string.split(",")[1])))
        new_image = myPillow.force_decrease_image_file_size(image, 10)
        return base64.b64encode(new_image.read())
    except Exception as e:
        print(base64string)
        raise(e)

In [5]:
import os
import json
import sqlite3

## export data to json

In [6]:
db_file = os.path.abspath("./mine.db")

class Self():
    pass

self = Self()
self.SQL_DATA_FILE = db_file
if not os.path.exists(self.SQL_DATA_FILE):
    print(f"SQL doesn't exist: {self.SQL_DATA_FILE}")
    print(f"We'll create a new one!")
self._sql_conn = sqlite3.connect(self.SQL_DATA_FILE, check_same_thread=False)
self._sql_cursor = self._sql_conn.cursor()

days = []
for row in self._sql_cursor.execute('SELECT * FROM messages ORDER BY date'):
    if " " in row[0]:
        if "." in row[0]:
            date = row[0].split(".")[0]
        else:
            date = row[0]
        row = [date, row[1], row[2]]
        
        #images = json.loads(row[2])
        #new_image_bytes_list = [myCompress(image) for image in images]
        #new_stringImage_list = [str(base64.b64encode(i))[2:-1] for i in new_image_bytes_list]
        #row[2] = json.dumps(new_stringImage_list)
        #print(len(stringImage), len(new_stringImage))
        row[2] = json.dumps([element.split(",")[1] for element in json.loads(row[2])])
        days.append({
            "date": row[0],
            "content": row[1],
            "images": row[2]
        })

text = json.dumps(days, indent=4)
with open("mine.json", "w") as f:
    f.write(text)

In [7]:
days[-1]

{'date': '2021-08-21 10:43:23',
 'content': 'The more I work for my company. \n\nThe more I feel offline APP is gonna be dying (hard to make profit from).',
 'images': '[]'}

## export data to database

In [15]:
with open("mine.json", "r") as f:
    text = f.read()
days = json.loads(text)
days = [[day[0]] + day[2:] for day in days]

#days[0]

In [8]:
!rm mine.db

In [9]:
class Self():
    pass

self = Self()
self.SQL_DATA_FILE = "./mine.db"
if not os.path.exists(self.SQL_DATA_FILE):
    print(f"SQL doesn't exist: {self.SQL_DATA_FILE}")
    print(f"We'll create a new one!")
self._sql_conn = sqlite3.connect(self.SQL_DATA_FILE, check_same_thread=False)
self._sql_cursor = self._sql_conn.cursor()
self._sql_cursor.execute('''CREATE TABLE IF NOT EXISTS messages
        (date TEXT, content TEXT, images TEXT)''')

for day in days:
    self._sql_cursor.execute("INSERT INTO messages VALUES (?,?,?)", day)
    self._sql_conn.commit()

SQL doesn't exist: ./mine.db
We'll create a new one!
